In [6]:
import mediapipe as mp
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import tqdm

import numpy as np
import pathlib
import os
import random

objc[98603]: Class CaptureDelegate is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1647f0860) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x338712480). One of the two will be used. Which one is undefined.
objc[98603]: Class CVWindow is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x15924ca68) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x3387124d0). One of the two will be used. Which one is undefined.
objc[98603]: Class CVView is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x15924ca90) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x3387124f8). One of the two will be used. Which

In [7]:
mp_holistic = mp.solutions.holistic # get holistic model
mp_drawing = mp.solutions.drawing_utils # get drawing tools

In [8]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert image color to RGB
    image.flags.writeable = False                  # don't show this image in feed
    results = model.process(image)                 # make holistics prediction
    image.flags.writeable = True                   # make image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert back to BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, lh, rh])

In [9]:
labels_path = tf.keras.utils.get_file(
    fname='labels.txt',
    origin='https://raw.githubusercontent.com/tensorflow/models/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/kinetics_600_labels.txt'
)
labels_path = pathlib.Path(labels_path)

lines = labels_path.read_text().splitlines()
KINETICS_600_LABELS = np.array([line.strip() for line in lines])
KINETICS_600_LABELS[:20]

array(['abseiling', 'acting in play', 'adjusting glasses', 'air drumming',
       'alligator wrestling', 'answering questions', 'applauding',
       'applying cream', 'archaeological excavation', 'archery',
       'arguing', 'arm wrestling', 'arranging flowers',
       'assembling bicycle', 'assembling computer',
       'attending conference', 'auctioning', 'backflip (human)',
       'baking cookies', 'bandaging'], dtype='<U49')

In [ ]:
# DATA_PATH = 'experiment_subset'
# subset_paths = {'train': pathlib.Path(os.path.join(DATA_PATH, 'train')),
#                 'val': pathlib.Path(os.path.join(DATA_PATH, 'val')),
#                 'test': pathlib.Path(os.path.join(DATA_PATH, 'test'))}
# print(subset_paths)

In [ ]:
# def frames_from_video_file(video_path, n_frames, output_size = (256,256)):
#   """ Creates frames from each video file present for each category.

#     Args:
#       video_path: File path to the video.
#       n_frames: Number of frames to be created per video file.
#       output_size: Pixel size of the output frame image.

#     Return:
#       An NumPy array of frames in the shape of (n_frames, height, width, channels).
#   """
#   # Read each frame by frame
#   result = []
#   src = cv2.VideoCapture(str(video_path))  

#   video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

#   # If the number of frames wanted is greater than the length of the video, then start from beginning
#   if n_frames > video_length:
#     start = 0
#   else:
#     # Otherwise, start at another random point within the video
#     max_start = video_length - n_frames
#     start = random.randint(0, max_start)

#   src.set(cv2.CAP_PROP_POS_FRAMES, start)

#   for _ in range(n_frames):
#     ret, frame = src.read()
#     if ret:
#       frame = tf.image.convert_image_dtype(frame, tf.float32)
#       frame = tf.image.resize_with_pad(frame, *output_size)
#       result.append(frame)
#     else:
#       result.append(np.zeros_like(result[0]))
#   src.release()
#   # Ensure that the color scheme is not inverted
#   result = np.array(result)[..., [2, 1, 0]]

#   return result

# class FrameGenerator:
#   def __init__(self, path, n_frames):
#     """ Returns a set of frames with their associated label. 

#       Args:
#         path: Video file paths.
#         classes: List of labels for classification.
#     """
#     self.path = path
#     self.n_frames = n_frames
#     self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
#     self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

#   def get_files_and_class_names(self):
#     video_paths = list(self.path.glob('*/*.mp4'))
#     classes = [p.parent.name for p in video_paths] 
#     return video_paths, classes

#   def __call__(self):
#     video_paths, classes = self.get_files_and_class_names()

#     pairs = list(zip(video_paths, classes))

#     random.shuffle(pairs)

#     for path, name in pairs:
#       video_frames = frames_from_video_file(path, self.n_frames) 
#       label = self.class_ids_for_name[name] # Encode labels
#       yield video_frames, label
    
# def get_top_k(probs, k=5, label_map=KINETICS_600_LABELS):
#   """Outputs the top k model labels and probabilities on the given video.

#   Args:
#     probs: probability tensor of shape (num_frames, num_classes) that represents
#       the probability of each class on each frame.
#     k: the number of top predictions to select.
#     label_map: a list of labels to map logit indices to label strings.

#   Returns:
#     a tuple of the top-k labels and probabilities.
#   """
#   # Sort predictions to find top_k
#   top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
#   # collect the labels of top_k predictions
#   top_labels = tf.gather(label_map, top_predictions, axis=-1)
#   # decode lablels
#   top_labels = [label.decode('utf8') for label in top_labels.numpy()]
#   # top_k probabilities of the predictions
#   top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
#   return tuple(zip(top_labels, top_probs))

In [ ]:
# output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
#                     tf.TensorSpec(shape = (), dtype = tf.int16))
# train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], 12),
#                                           output_signature = output_signature)
# train_frames, train_labels = next(iter(train_ds))
# print(f'Shape of training set of frames: {train_frames.shape}')
# print(f'Shape of training labels: {train_labels.shape}')

In [ ]:
# %%time
# id = 'a0'
# mode = 'stream'
# version = '3'
# hub_url = f'https://tfhub.dev/tensorflow/movinet/{id}/{mode}/kinetics-600/classification/{version}'
# model = hub.load(hub_url)

In [ ]:
# lines = model.signatures['init_states'].pretty_printed_signature().splitlines()
# lines = lines[:10]
# lines.append('      ...')
# print('.\n'.join(lines))

In [ ]:
# init_states = model.init_states(train_frames[tf.newaxis].shape)

In [ ]:
# # Insert your video clip here
# video = train_frames
# images = tf.split(video[tf.newaxis], video.shape[0], axis=1)

# all_logits = []

# # To run on a video, pass in one frame at a time
# states = init_states
# for image in tqdm.tqdm(images):
#   # predictions for each frame
#   logits, states = model({**states, 'image': image})
#   all_logits.append(logits)

# # concatinating all the logits
# logits = tf.concat(all_logits, 0)
# # estimating probabilities
# probs = tf.nn.softmax(logits, axis=-1)

In [ ]:
# final_probs = probs[-1]
# print('Top_k predictions and their probablities\n')
# for label, p in get_top_k(final_probs):
#   print(f'{label:20s}: {p:.3f}')

In [1]:
# import tensorflow as tf
# import tensorflow_hub as hub

#from official.projects.movinet.modeling import movinet
#from official.projects.movinet.modeling import movinet_model

In [ ]:
# model_id = 'a0'

# tf.keras.backend.clear_session()

# # Create backbone and model.
# use_positional_encoding = model_id in {'a3', 'a4', 'a5'}
# backbone = movinet.Movinet(
#     model_id=model_id,
#     causal=True,
#     conv_type='2plus1d',
#     se_type='2plus3d',
#     activation='hard_swish',
#     gating_activation='hard_sigmoid',
#     use_positional_encoding=use_positional_encoding,
#     use_external_states=True,
# )

# # Create a movinet classifier using this backbone.
# model = movinet_model.MovinetClassifier(
#     backbone,
#     num_classes=3,
#     output_states=True)

In [2]:
# model_id = 'a0'

# tf.keras.backend.clear_session()

# movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/stream/kinetics-600/classification/3'

# movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)

# pretrained_weights = {w.name: w for w in movinet_hub_model.weights}

# model_weights = {w.name: w for w in model.weights}

# for name in pretrained_weights:
#     model_weights[name].assign(pretrained_weights[name])

In [3]:
# # Input layer for the frame sequence
# image_input = tf.keras.layers.Input(
#     shape=[None, None, None, 3],
#     dtype=tf.float32,
#     name='image')

# # Input layers for the different model states.
# init_states_fn = movinet_hub_model.resolved_object.signatures['init_states']


# state_shapes = {
#     name: ([s if s > 0 else None for s in state.shape], state.dtype)
#     for name, state in init_states_fn(tf.constant([0, 0, 0, 0, 3])).items()
# }

# states_input = {
#     name: tf.keras.Input(shape[1:], dtype=dtype, name=name)
#     for name, (shape, dtype) in state_shapes.items()
# }

# # Wrap the Movinet model in a Keras model so that it can be finetuned.

# inputs = {**states_input, 'image': image_input}

# outputs = model(inputs)

2022-10-17 18:18:54.208372: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-17 18:18:54.300634: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x301caa9b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-10-17 18:18:54.300651: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2022-10-17 18:18:54.359344: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


NameError: name 'model' is not defined

In [ ]:
# class CustomModel(tf.keras.Model):
#     def train_step(self, data):
#         # Unpack the data. Its structure depends on your model and
#         # on what you pass to `fit()`.
#         if len(data) == 3:
#             x, y, sample_weight = data
#         else:
#             sample_weight = None
#             x, y = data
#         with tf.GradientTape() as tape:
#             pred, states = self(x, training=True)  # Forward pass
            
#             # Compute the loss value
#             # (the loss function is configured in `compile()`)
#             loss = self.compiled_loss(y, pred, regularization_losses=self.losses, sample_weight=sample_weight)

#         # Compute gradients
#         trainable_vars = self.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)
        
#         # Update weights
#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
#         # Update metrics (includes the metric that tracks the loss)
#         self.compiled_metrics.update_state(y, pred, sample_weight=sample_weight)
        
#         # Return a dict mapping metric names to current value
#         return {m.name: m.result() for m in self.metrics}


In [ ]:
# model = CustomModel(inputs, outputs, name='movinet')

In [ ]:
# for layer in model.layers[:-1]:
#     layer.trainable = False
# model.layers[-1].trainable = True

In [ ]:
# model.summary()

In [ ]:
# init_states = init_states_fn(tf.shape(tf.ones([BATCH_SIZE, NBFRAME, SIZE[0], SIZE[1], CHANNELS])))
# finetune data in the format ({**init_states, 'image': frame_sequence}, label)

In [10]:
hub_url = "https://tfhub.dev/tensorflow/movinet/a0/stream/kinetics-600/classification/3"

encoder = hub.KerasLayer(hub_url, trainable=False)

# Define the image (video) input
image_input = tf.keras.layers.Input(
    shape=[None, None, None, 3],
    dtype=tf.float32,
    name='image')

# Define the state inputs, which is a dict that maps state names to tensors.
init_states_fn = encoder.resolved_object.signatures['init_states']
state_shapes = {
    name: ([s if s > 0 else None for s in state.shape], state.dtype)
    for name, state in init_states_fn(tf.constant([0, 0, 0, 0, 3])).items()
}
states_input = {
    name: tf.keras.Input(shape[1:], dtype=dtype, name=name)
    for name, (shape, dtype) in state_shapes.items()
}

# The inputs to the model are the states and the video
inputs = {**states_input, 'image': image_input}

outputs = encoder(inputs)

model = tf.keras.Model(inputs, outputs, name='movinet')

In [11]:
# Create your example input here.
# Refer to the description or paper for recommended input shapes.
example_input = tf.random.normal([1, 8, 172, 172, 3])

# Split the video into individual frames.
# Note: we can also split into larger clips as well (e.g., 8-frame clips).
# Running on larger clips will slightly reduce latency overhead, but
# will consume more memory.
frames = tf.split(example_input, example_input.shape[1], axis=1)

# Initialize the dict of states. All state tensors are initially zeros.
init_states = init_states_fn(tf.shape(example_input))

# Run the model prediction by looping over each frame.
states = init_states
predictions = []
for frame in frames:
    output, states = model({**states, 'image': frame})
    predictions.append(output)

# The video classification will simply be the last output of the model.
final_prediction = tf.argmax(predictions[-1], -1)

# Alternatively, we can run the network on the entire input video.
# The output should be effectively the same
# (but it may differ a small amount due to floating point errors).
#non_streaming_output, _ = model({**init_states, 'image': example_input})
#non_streaming_prediction = tf.argmax(non_streaming_output, -1)

tf.Tensor([6], shape=(1,), dtype=int64)
['applauding']


In [28]:
sequence = []
sentence = []
out = np.array([0])
threshold = 0
num_frames = 8

cap = cv2.VideoCapture(0)
# access mp model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # read feed
        ret, frame = cap.read()

        # get holistic detections
        image, results = mp_detection(frame, holistic)
        
        # draw landmarks
        draw_landmarks(image, results)
        
        frame = cv2.resize(frame, (172, 172))/255

        #keypoints = extract_keypoints(results)
        sequence.append(frame)
        #sequence = tf.concat([sequence, frame], 0)
        sequence = sequence[-num_frames:]
        #print(tf.convert_to_tensor(sequence).shape)

        if len(sequence) == num_frames:
            #out = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(out)])
            batch = tf.convert_to_tensor(sequence, dtype = 'float32')[tf.newaxis]
            #out = infer(batch[tf.newaxis])['dense_5'].numpy()[0]
            frames = tf.split(batch, batch.shape[1], axis=1)
            init_states = init_states_fn(tf.shape(batch))
            states = init_states
            predictions = []
            for f in frames:
                output, states = model({**states, 'image': f})
                predictions.append(output)
                
            out = list(KINETICS_600_LABELS[tf.argmax(predictions[-1], -1)])
            
            sentence += out
            if len(sentence) > 5:
                sentence = sentence[-5:]
                
            

#         if out[np.argmax(out)] > threshold:
#             if len(sentence) > 0:
#                 if actions[np.argmax(out)] != sentence[-1]:
#                     sentence.append(actions[np.argmax(out)])
#             else:
#                 sentence.append(actions[np.argmax(out)])

#         if len(sentence) > 5:
#             sentence = sentence[-5:]
        
            print(' '.join(sentence))
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        

        # show to screen
        cv2.imshow('OpenCV Feed', image)

        # press q to break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

beatboxing
beatboxing playing didgeridoo
beatboxing playing didgeridoo playing harmonica
beatboxing playing didgeridoo playing harmonica playing trumpet
beatboxing playing didgeridoo playing harmonica playing trumpet playing trumpet
playing didgeridoo playing harmonica playing trumpet playing trumpet playing didgeridoo
playing harmonica playing trumpet playing trumpet playing didgeridoo playing didgeridoo
playing trumpet playing trumpet playing didgeridoo playing didgeridoo playing didgeridoo
playing trumpet playing didgeridoo playing didgeridoo playing didgeridoo playing didgeridoo
playing didgeridoo playing didgeridoo playing didgeridoo playing didgeridoo playing didgeridoo
playing didgeridoo playing didgeridoo playing didgeridoo playing didgeridoo staring
playing didgeridoo playing didgeridoo playing didgeridoo staring staring
playing didgeridoo playing didgeridoo staring staring staring
playing didgeridoo staring staring staring staring
staring staring staring staring playing didge

playing pan pipes playing pan pipes playing pan pipes playing pan pipes cosplaying
playing pan pipes playing pan pipes playing pan pipes cosplaying cosplaying
playing pan pipes playing pan pipes cosplaying cosplaying talking on cell phone
playing pan pipes cosplaying cosplaying talking on cell phone playing harmonica
cosplaying cosplaying talking on cell phone playing harmonica playing harmonica
cosplaying talking on cell phone playing harmonica playing harmonica playing harmonica
talking on cell phone playing harmonica playing harmonica playing harmonica smoking pipe
playing harmonica playing harmonica playing harmonica smoking pipe smoking hookah
playing harmonica playing harmonica smoking pipe smoking hookah smoking hookah
playing harmonica smoking pipe smoking hookah smoking hookah staring
smoking pipe smoking hookah smoking hookah staring staring
smoking hookah smoking hookah staring staring staring
smoking hookah staring staring staring staring
staring staring staring staring sta

In [30]:
DATA_PATH = 'experiment_subset'
subset_paths = {'train': pathlib.Path(os.path.join(DATA_PATH, 'train')),
                'val': pathlib.Path(os.path.join(DATA_PATH, 'val')),
                'test': pathlib.Path(os.path.join(DATA_PATH, 'test'))}
print(subset_paths)

{'train': PosixPath('experiment_subset/train'), 'val': PosixPath('experiment_subset/val'), 'test': PosixPath('experiment_subset/test')}


In [31]:
class_names = sorted(set(p.name for p in pathlib.Path(os.path.join(DATA_PATH, 'train')).iterdir() if p.is_dir()))
label_map = dict((name, idx) for idx, name in enumerate(class_names))
actions = list(label_map.keys())

{'hello': 0, 'love': 1, 'thank you': 2}